<a href="https://colab.research.google.com/github/tchtinku/DALLE-mini/blob/main/DALLE_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installation & Setup

In [2]:
!pip install jax==0.3.25 jaxlib==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

!pip install -q dalle-mini
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 MB 1.8 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.3.25-py3-none-any.whl size=1308492 sha256=86b2582e4e39938def29924bf7239ed6f69cfe95ca0a84e963e6b6eb906bc861
  Stored in directory: /root/.cache/pip/wheels/41/0b/45/1acffcaf4c863a6d0d0e910e56ae6502ca8d16300e39e1dab2
Successfully built jax
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.23+cuda12.cudnn89
    Uninstalling jaxlib-0.4.23+cuda12.cudnn89:
      Successfully uninstalled jaxlib-0.4.23+cuda12.cudnn89
  Attempting uninstall: jax
    Found existing installation: jax 0.4.23
    Uninstalling jax-0.4.23:
      Successfully uninstalled jax-0.4.23
ERROR: pip's dependency resolver does not currently take into account all the packages that a

###We load required models:
DALL·E mini for text to encoded images
VQGAN for decoding images
CLIP for scoring predictions

In [3]:
DALLE_MODEL = "dalle-mini/dalle-mini/mega-1-fp16:latest"
DALLE_COMMIT_ID = None

VOGAN_REPO =  "dalle-mini/vqgan_imagenet_f16_16384"
VOGAN_COMMIT_ID = "e93a26e7707683d349bf5d5c41c5b0ef69b677a9"

In [4]:
!pip install dalle-mini
!pip install vqgan-jax
from vqgan_jax.modeling_flax_vqgan import VQModel

In [5]:
import jax
import jax.numpy as jnp

jax.local_device_count()

1

In [6]:
# Install required packages.
!pip install dalle-mini==0.6.0
!pip install vqgan-jax==0.3.0
!pip install flax
!pip install git+https://github.com/EleutherAI/vqgan-jax.git
!pip install numpy
!pip install jax
!pip install optax
!pip install vqgan-jax

# Import required modules.
from dalle_mini import DalleBart, DalleBartProcessor
from vqgan_jax.modeling_flax_vqgan import VQModel
from transformers import FlaxCLIPModel, CLIPProcessor

# Load the DalleBart model and parameters.
model, params = DalleBart.from_pretrained(
    "dalle-mini/dalle-mini/3b-float16", revision="fp16", dtype=jnp.float16, _do_init=False
)

# Load the VQGAN model and parameters.
vqgan, vqgan_params = VQModel.from_pretrained(
    "dalle-mini/vqgan_40M", revision="fp16", _do_init=False
)

ERROR: Could not find a version that satisfies the requirement dalle-mini==0.6.0 (from versions: 0.0.1.dev1, 0.0.2, 0.0.3, 0.0.5, 0.0.6, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5)
ERROR: No matching distribution found for dalle-mini==0.6.0
ERROR: Could not find a version that satisfies the requirement vqgan-jax==0.3.0 (from versions: none)
ERROR: No matching distribution found for vqgan-jax==0.3.0
  Cloning https://github.com/EleutherAI/vqgan-jax.git to /tmp/pip-req-build-5risj5u8
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/vqgan-jax.git /tmp/pip-req-build-5risj5u8
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/EleutherAI/vqgan-jax.git /tmp/pip-req-build-5risj5u8 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a prob

OSError: Can't load the configuration of 'dalle-mini/dalle-mini/3b-float16'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'dalle-mini/dalle-mini/3b-float16' is the correct path to a directory containing a config.json file

Model parameters are replicated on each device for faster inference.

In [ ]:
from flax.jax_utils import replicate

params = replicate(params)
vqgan_params = replicate(vqgan_params)

NameError: name 'params' is not defined

Model functions are compiled and parallelized to take advantage of multiple devices.

In [ ]:
from functools import partial

@partial(jax.pmap, axis_name="batch", static_broadcasted_argnums=(3,4,5,6))
def p_generate(
    tokenized_prompt, key, params, top_k, top_p, temperature, condition_scale
):
return model.generate(
    **tokenized_prompt,
    prng_key=key,
    params=params,
    top_k=top_k,
    top_p=top_p,
    temperature=temperature,
    condition_scale=condition_scale
)

@partial(jax.pmap, axis_name="batch")
def p_decode(indices, params):
  return vqgan.decode_code(indices, params=params)

Keys are passed to the model on each device to generate unique inference per device

In [ ]:
import random

seed = random.randint(0, 2**32 - 1)
key = jax.random.PRNGKey(seed)

##Text

our model requires proceeding prompts

In [ ]:
from dalle_mini import DalleBartProcessor

processor = DalleBartProcessor.from_pretrained(DALLE_MODEL, revision=DALLE_COMMIT_ID)

let's define some text prompts

In [ ]:
prompts =  [
    "Sunshine over a lake in the mountains",
    "the Eiffel tower landing on the moon"
]

Note: we could use the same prompt multiple times for faster inference

In [ ]:
tokenized_prompts = processor(prompts)

finally we replicate prompts onto each device

In [ ]:
tokenized_prompts = replicate(tokenized_prompts)

##Generate Images

we generate images using dalle-mini model and decode them with VQGAN

In [ ]:
n_predictions=8

gen_top_k = None
gen_top_p = None
temperature = None
cond_scale = 10.0

In [ ]:
from flax.training.common_utils import shard_prng_key
import numpy as np
from PIL import image
from tqdm.notebook import trange

print(f"Prompts: {prompts}\n")
images=[]
for i in trange(max(n_predictions // jax.device_count(), 1)):
  key, subkey = jax.random.split(key)
  encoded_images = p_generate(
      tokenized_prompt,
      shard_prng_key(subkey),
      params,
      gen_top_k,
      gen_top_p,
      temperature,
      cond_scale

  )
  encoded_images=encoded_images.sequences[..., 1]
  decoded_images=p_decode(encoded_images, vqgan_params)
  decoded_images=decoded_images.clip(0.0, 1.0).reshape((-1, 256, 256, 3))
  for decoded_img in decoded_images:
    img = Image.fromarray(np.asarray(decoded_img * 255, dtype=np.uint8))
    images.append(img)
    display(img)
    print()

#Optional rank images by CLIP Score

We can rank images according to CLIP

NOTE: Sessin may crash because of unavilability of Colab pro

In [ ]:
CLIP_REPO = "openai/clip-vit-base-patch32"
CLIP_COMMIT_ID = None

clip, clip_params = FlaxCLIPModel.from_pretrained(
    CLIP_REPO, revision=CLIP_COMMIT_ID, dtype=jnp.float16, _do_init=False
)
clip_processor = CLIPPROCESSOR.from_pretrained(CLIP_REPO, revision=CLIP_COMMIT_ID)
clip_params = replicate(clip_params)

@partial(jax.pmap, axis_name="batch")
def p_clip(inputs, params):
  logits = clip(params=params, **inputs).logits_per_image
  return logits

In [ ]:
from flax.training.common_utils import shard

clip_inputs = clip_processor(
    text=prompts*jax.device_count(),
    images=images,
    return_tensors="np",
    padding="max_length",
    max_length=77,
    truncation=True,
).data
logits=p_clip(shard(clip_inputs), clip_params)

p=len(prompts)
logits=np.asarray([logits[:, i::p, i] for i in range(p)]).squeeze()

Let's now display images ranked by CLIP score

In [ ]:
for i, prompt in enumerate(prompts):
  print(f"Prompt: {prompt}\n")
  for idx in logits[i].argsort()[::-1]:
    display(images[idx*p+i])
    print(f"Score: {jnp.asarray(logits[i][idx], dtype=jnp.float32):.2f}\n")

#Save your images as W&B tables

W&B Table is an interactive 2D grid with support to rich media logging. Use this to save the generated images on W&B dashboard and share with the world



In [ ]:
import wandb

project="dalle-mini-tables-colab"
run = wandb.init(project=project)

columns=["captions"] + [f"image_{i+1}" for i in range(n_predictions)]
gen_table=wandb.Table(columns=columns)

for i, prompt in enumerate(prompts):
  if logits is not None:
    idxs= logits[i].argsort()[::-1]
    tmp_imgs = images[i :: len(prompts)]
    tmp_imgs = [tmp_imgs[idx] for idx in idxs]
  else:
    tmp_imgs = images[i :: len(prompts)]


  gen_table.add_data(prompt, *[wandb.Image(img) for img in tmp_imgs])

wandb.log({"Generated Images": gen_table})

run.finish()